In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [3]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = api_key)

In [4]:
query = "Hi there, I have a question about my bill. Can you help me?"

messages = [
    {
        "role": "system",
        "content": """
        You're a helpful customer care assistant that can classify incoming messages and create a response.
        Always response in the following JSON format: {"content": <response>, "category": <classification>}
        Available categories: 'general', 'order', 'billing'
        """,
    },
    {
        "role": "user",
        "content": query,
    },
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    response_format={"type": "json_object"},
)
message = response.choices[0].message.content



In [5]:
print(message)

{"content": "Of course! I'd be happy to help you with your billing question. Could you please provide more details about your issue?", "category": "billing"}


In [6]:
type(message)  # str

str

In [7]:
message_json = json.loads(message)
type(message_json)  # dict

dict

In [34]:
query = """
Hi, I'm having trouble with my recent order. I received the wrong item and need to return it for a refund. 
Can you help me with the return process and let me know when I can expect my refund?
"""

# --------------------------------------------------------------
# Providing a JSON Schema with detailed parameter descriptions
# --------------------------------------------------------------

system_prompt = """
You are an AI customer care assistant. You will be provided with a customer inquiry,
and your goal is to respond with a structured solution, including the steps taken to resolve the issue and the final resolution.
For each step, provide a detailed description and the action taken.
"""

def get_ticket_response_json(query):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ticket_resolution",
                "schema": {
                    "type": "object",
                    "properties": {
                        "steps": {
                            "type": "array",
                            "description": (
                                "An ordered list of steps taken to resolve the customer's inquiry. "
                                "Each step is an object containing a detailed description and the specific action performed."
                            ),
                            "items": {
                                "type": "object",
                                "description": "A single step in the resolution process.",
                                "properties": {
                                    "description": {
                                        "type": "string",
                                        "description": "A detailed explanation of the step taken."
                                    },
                                    "action": {
                                        "type": "string",
                                        "description": "The specific action performed in this step."
                                    }
                                },
                                "required": ["description", "action"],
                                "additionalProperties": False
                            }
                        },
                        "final_resolution": {
                            "type": "string",
                            "description": (
                                "A summary of the final resolution provided to the customer, "
                                "including any instructions or expected outcomes."
                            )
                        }
                    },
                    "required": ["steps", "final_resolution"],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        },
    )

    return response.choices[0].message


In [35]:
response = get_ticket_response_json(query)


In [36]:
res = json.loads(response.content)

In [37]:
res

{'steps': [{'description': 'Verify the order information and the items received by the customer to confirm if they do not match the original order.',
   'action': "Reviewed the customer's order details and confirmed that the item received was indeed incorrect."},
  {'description': 'Initiate a return process by generating a return label for the incorrect item and providing instructions for the customer',
   'action': 'Created a return shipping label for the customer and emailed it to them along with detailed return instructions.'},
  {'description': 'Inform the customer of the expected timeline for processing the return and issuing the refund once the item is received.',
   'action': 'Explained to the customer that upon receipt of the returned item, the refund process will begin, typically taking 5-7 business days to reflect in their payment method.'}],
 'final_resolution': 'I have sent you a return shipping label via email. Please use this label to return the incorrect item. Once recei

In [38]:
res['steps']

[{'description': 'Verify the order information and the items received by the customer to confirm if they do not match the original order.',
  'action': "Reviewed the customer's order details and confirmed that the item received was indeed incorrect."},
 {'description': 'Initiate a return process by generating a return label for the incorrect item and providing instructions for the customer',
  'action': 'Created a return shipping label for the customer and emailed it to them along with detailed return instructions.'},
 {'description': 'Inform the customer of the expected timeline for processing the return and issuing the refund once the item is received.',
  'action': 'Explained to the customer that upon receipt of the returned item, the refund process will begin, typically taking 5-7 business days to reflect in their payment method.'}]

In [39]:
res['final_resolution']

'I have sent you a return shipping label via email. Please use this label to return the incorrect item. Once received, we will process your refund, which should reflect in your payment method within 5-7 business days.'